<a href="https://colab.research.google.com/github/nhanlunlk2k/APP-project/blob/main/app_group7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import copy

In [2]:
def find_min_max_2D(M):
    min = M[0][0]
    max = M[0][0]
    for i in range(len(M)):
        for j in range(len(M[i])):
            if M[i][j] < min:
                min = M[i][j]
            elif M[i][j] > max:
                max = M[i][j]
    return min,max

def find_mean_2D(M):
    sum = 0
    for i in range(len(M)):
        for j in range(len(M[i])):
            sum += M[i][j]
    return sum/(len(M)*len(M[0]))

def find_median_2D(M):
    tmp = copy.copy(M[0])
    for i in range(1,len(M)):
        tmp.extend(M[i])
    tmp.sort()
    if len(tmp)%2 == 0:
        return (tmp[len(tmp)//2] + tmp[len(tmp)//2 - 1])/2
    else:
        return tmp[len(tmp)//2]

def pad_2D(M,padding_size,mode):
    num_cols = len(M[0]) + padding_size[0][0] + padding_size[0][1]
    num_rows = len(M) + padding_size[1][0] + padding_size[1][1]
    res = []
    for i in range(num_rows):
        row = []
        for j in range(num_cols):
            row.append(0)
        res.append(row)
    for i in range(len(M)):
        for j in range(len(M[i])):
            res[i+padding_size[1][0]][j+padding_size[0][0]] = M[i][j]
    match mode:
        case 'min':
            cons = find_min_max_2D(M)[0]
            for i in range(len(res)):
                for j in range(padding_size[0][0]):
                    res[i][j]=cons
                for j in range(1,padding_size[0][1]+1):
                    res[i][-j]=cons
            line = [cons]*num_cols
            for j in range(padding_size[1][0]):
                res[j] = [cons]*num_cols
            for j in range(1,padding_size[1][1] + 1):
                res[-j] = [cons]*num_cols
            return res
        case 'max':
            cons = find_min_max_2D(M)[1]
            for i in range(len(res)):
                for j in range(padding_size[0][0]):
                    res[i][j]=cons
                for j in range(1,padding_size[0][1]+1):
                    res[i][-j]=cons
            line = [cons]*num_cols
            for j in range(padding_size[1][0]):
                res[j] = [cons]*num_cols
            for j in range(1,padding_size[1][1]+1):
                res[-j] = [cons]*num_cols
            return res
        case 'mean':
            cons = find_mean_2D(M)
            for i in range(len(res)):
                for j in range(padding_size[0][0]):
                    res[i][j]=cons
                for j in range(1,padding_size[0][1]+1):
                    res[i][-j]=cons
            for j in range(padding_size[1][0]):
                res[j] = [cons]*num_cols
            for j in range(1,padding_size[1][1]+1):
                res[-j] = [cons]*num_cols
            return res
        case 'zero':
            return res
        case 'median':
            cons = find_median_2D(M)
            for i in range(len(res)):
                for j in range(padding_size[0][0]):
                    res[i][j]=cons
                for j in range(1,padding_size[0][1]+1):
                    res[i][-j]=cons
            for j in range(padding_size[1][0]):
                res[j] = [cons]*num_cols
            for j in range(1,padding_size[1][1]+1):
                res[-j] = [cons]*num_cols
            return res
        case 'edge':
            r = list(range(-padding_size[1][0],len(M) + padding_size[1][1]))
            c = list(range(-padding_size[0][0],len(M[0]) + padding_size[0][1]))
            res_id = []
            for i in range(len(res)):
                row = []
                for j in range(len(res[i])):
                    if r[i] < 0:
                        r[i] = 0
                    elif r[i] >= len(M):
                        r[i] = len(M) - 1
                    if c[j] < 0:
                        c[j] = 0
                    elif c[j] >= len(M[0]):
                        c[j] = len(M[0]) - 1
                    row.append((r[i],c[j]))
                res_id.append(row)
            for i in range(len(res)):
                for j in range(len(res[i])):
                    res[i][j] = M[res_id[i][j][0]][res_id[i][j][1]]
            return res
        case _:
            raise ValueError('Invalid mode')

In [3]:
def num_dim(A):
    if type(A) == list:
        return 1 + num_dim(A[0])
    else:
        return 0

def array_shape(A):
    shape = []
    while type(A) == list:
        shape.append(len(A))
        A = A[0]
    return shape

#convolution using Python
def convolution_py(F,M,padding=None): #F is a filters (n filters x 2d array), M is a volume (3d array)
    F_size = array_shape(F)
    up_padding = F_size[1]//2
    down_padding = F_size[1] - F_size[1]//2 - 1
    left_padding = F_size[2]//2
    right_padding = F_size[2] - F_size[2]//2 - 1
    pad_size = ((left_padding,right_padding),(up_padding,down_padding))
    res = []
    M_copy = copy.copy(M)
    if padding != None:
        for i in range(len(M)):
            M_copy[i] = pad_2D(M[i],pad_size,padding)
        res = []
        for k in range(len(F)):
            m = []
            for i in range(len(M[0])):
                row = []
                for j in range(len(M[0][0])):
                    row.append(0)
                m.append(row)
            res.append(m)
    else:
        res = []
        for k in range(len(F)):
            m = []
            for i in range(len(M[0]) - F_size[1] + 1):
                row = []
                for j in range(len(M[0][0]) - F_size[2] + 1):
                    row.append(0)
                m.append(row)
            res.append(m)
    #print(array_shape(res))
    M_size = array_shape(M_copy)
    #print(M_size)
    tmp = []
    for i in range(M_size[1]):
        row = []
        for j in range(M_size[2]):
            s = 0
            for k in range(M_size[0]):
                s += M_copy[k][i][j]
            row.append(s)
        tmp.append(row)
    #print(array_shape(tmp))
    for k in range(len(res)):
        for i in range(len(res[k])):
            for j in range(len(res[k][i])):
                for i_f in range(len(F[k])):
                    for j_f in range(len(F[k][i_f])):
                        res[k][i][j] += tmp[i+i_f][j+j_f]*F[k][i_f][j_f]
    return res

def pooling_py(M,pool_size,type_,stride): #M has n matrices
    res = []
    M_size = array_shape(M)
    for k in range(M_size[0]):
        res.append([])
    for k in range(M_size[0]):
        m = []
        for i in range(0,M_size[1],stride[0]):
            row = []
            for j in range(0,M_size[2],stride[1]):
                tmp = []
                for i_p in range(stride[0]):
                    for j_p in range(stride[1]):
                        r = i + i_p
                        c = j + j_p
                        if (r > M_size[1] - 1) or (c > M_size[2] - 1):
                            continue
                        else:
                            tmp.append(M[k][r][c])
                match type_:
                    case 'max':
                        row.append(max(tmp))
                    case 'min':
                        row.append(min(tmp))
                    case 'mean':
                        row.append(sum(tmp)/len(tmp))
            m.append(row)
        res[k] = m
    return res

def unpooling_py(M,pool_size,stride):
    pass

def deconvolution_py(F,M):
    pass

def matrix_mul(A,B): #Multiply 2 matrices
    res = [[0]*len(B[0])]*len(A)
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                res += A[i][k]*B[k][j]
    return res

In [ ]:
A = np.random.randint(0,10,(3,100,100))
B = np.random.randint(0,10,(3,100,100))
A_py = A.tolist()
B_py = B.tolist()
F = np.random.randint(0,10,(32,3,3))/10
F_py = F.tolist()
A_py

In [ ]:
C = convolution_py(F_py,A_py,'edge')
C

In [ ]:
array_shape(C)

In [ ]:
P = pooling_py(C,(2,2),'max',(2,2))
P

In [ ]:
array_shape(P)

In [16]:
#Convolution using Numpy
def convolution_np(F,M,padding=None): #F is filters (n filters x 2d array), M is a volume (3d array)
    up_padding = F.shape[1]//2
    down_padding = F.shape[1] - F.shape[1]//2 - 1
    left_padding = F.shape[2]//2
    right_padding = F.shape[2] - F.shape[2]//2 - 1
    pad_size = ((left_padding,right_padding),(up_padding,down_padding))
    res = []
    if padding != None:
        match padding:
            case 'min':
                M = np.pad(M,pad_size,mode='minimum')
            case 'max':
                M = np.pad(M,pad_size,mode='maximum')
            case 'mean':
                M = np.pad(M,pad_size,mode='mean')
            case 'zero':
                M = np.pad(M,pad_size,mode='constant',constant_values=0)
            case 'median':
                M = np.pad(M,pad_size,mode='median')
            case 'edge':
                M = np.pad(M,pad_size,mode='edge')
            case _:
                raise ValueError('Invalid padding mode')
        res = np.zeros((F.shape[0],M.shape[1],M.shape[2]))
    else:
        res = np.zeros((F.shape[0],M.shape[1] - F.shape[1] + 1,M.shape[2] - F.shape[2] + 1))
    tmp = M.sum(axis=0)
    for k in range(res.shape[0]):
        for i in range(res.shape[1]):
            for j in range(res.shape[2]):
                res[k][i][j] += (F * tmp[i:i + F.shape[1],j:j + F.shape[2]]).sum(axis=0)[k]
    return res

def pooling_np(M,pool_size,type_,stride):
    res = np.zeros((M.shape[0],M.shape[1]//stride[0],M.shape[2]//stride[1]))
    for k in range(0,M.shape[0]):
        for i in range(0,M.shape[1],stride[0]):
            for j in range(0,M.shape[2],stride[1]):
                tmp = M[:,i:i + pool_size[0],j:j + pool_size[1]]
            match type_:
                case 'max':
                    res[k][i][j] = np.max(tmp,axis=0)[k]
                case 'min':
                    res[k][i][j] = np.min(tmp,axis=0)[k]
                case 'mean':
                    res[k][i][j] = np.mean(tmp,asix=0)[k]
    return res

def unpooling_np(M,pool_size,stride):
    pass

def deconvolution_np(F,M):
    pass

In [ ]:
convolution_np(F,A)